# 8760 Load Analysis 
## Purpose
The goal of this code is to take 8760 .csv files from electrical system and analyze when and where the load is not being met. It was specifically designed with the output from the HOMER microgrid software in mind. 

In [1]:
# import modules 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import os 
import pathlib
from glob  import glob

In [2]:
# define functions

def count_consecutive_groups(df):
    groups = (df['hour'].diff() != 1).cumsum()  # Identify consecutive groups
    return len(groups.unique()) 


In [ ]:
#load csv file file path 
dir_path = os.path.join(
    pathlib.Path.home(),
        'code-projects',
        'load-8760'
        )


data_files = glob(os.path.join(dir_path,'*.csv'))

#list to store all results

results = []
outages_summary=[]


for data_path in data_files:
    data = pd.read_csv(
        data_path,
        delimiter=',',
        header= 0,
        index_col='Time',
        skiprows=1)
     
    data= data[data.index.notnull()]

    data=data.reset_index() #Reset Index
    data["Time"] = pd.to_datetime(data["Time"])
    data["hour"] = data['Time'].map(lambda x: x.hour)
    data["day"] = data['Time'].map(lambda x: x.day)
    data["month"] = data['Time'].map(lambda x: x.month)
    

     #Initialize N/A
    hours_shortage = 'NA'
    hours_battery_shortage = 'NA'
    hours_unmet_load = 'NA' 
    num_days_outages = 'NA' 
    outages_year = 'NA' 
    # filter  how many hours load not met (capacity shortage > 0)
    if 'Capacity Shortage' in data.columns:
        data['Capacity Shortage'] = data['Capacity Shortage'].astype(float) #first need to force column to convert to float 
        capacity_shortage_df = data[data['Capacity Shortage'] > 0 ]
        hours_shortage = len(capacity_shortage_df)
   

    #Calculate how many hours annual load is unmet   #for Battery <30%
    if 'Generic 1kWh Li-Ion State of Charge' in data.columns:
        data['Generic 1kWh Li-Ion State of Charge'] = data['Generic 1kWh Li-Ion State of Charge'].astype(float)
        battery_shortage_df = data[data['Generic 1kWh Li-Ion State of Charge'] < 30 ]
        hours_battery_shortage = len(battery_shortage_df)
    # To find hours with Unmet Electrical Load

    if 'Unmet Electrical Load' in data.columns:
        data['Unmet Electrical Load'] = data['Unmet Electrical Load'].astype(float)
        unmet_load_df = data[data['Unmet Electrical Load'] > 0.01 ]
        hours_unmet_load = len(unmet_load_df)
    
    hours_per_day = unmet_load_df.groupby(['month','day']).size().reset_index()
    hours_per_day.columns=['month','day','hours_with_unmet_load']
    #Calculate number of days with outages 
    num_days_outages = len(hours_per_day)
    # Calulate number of outages per year
    outages_year = count_consecutive_groups (unmet_load_df)
    #Calculate number of distinct outages 
    outages_day = unmet_load_df.groupby(['month', 'day']).apply(count_consecutive_groups).reset_index()
    outages_day.columns = ['month','day','#_of_outages']
    #complile a list of results for summary 
    results.append ({
         'File': os.path.basename(data_path),
        'Hours with Capacity Shortage': hours_shortage,
        'Hours with Battery <30%': hours_battery_shortage,
        'Hours with Unmet Load': hours_unmet_load,
        'Days with Outages': num_days_outages,
        'Total Outages (Year)': outages_year
    })
    #Merge hours_per_day and outages_year data frames
    if not hours_per_day.empty and not outages_day.empty:
        merged = pd.merge(hours_per_day, outages_day, on=['month', 'day'], how='outer')
        merged['File'] = os.path.basename(data_path)
        outages_summary.append(merged)

results_df = pd.DataFrame(results)
outages_summary_df = pd.concat(outages_summary, ignore_index=True)



C:\Users\Nolan Welsh\AppData\Local\Temp\ipykernel_28748\73260671.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Time"] = pd.to_datetime(data["Time"])
C:\Users\Nolan Welsh\AppData\Local\Temp\ipykernel_28748\73260671.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  outages_day = unmet_load_df.groupby(['month', 'day']).apply(count_consecutive_groups).reset_index()
C:\Users\Nolan Welsh\AppData\Local\Temp\ipykernel_28748\73260671.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To 

                    File Hours with Capacity Shortage  \
0    0pctCapShortage.csv                           NA   
1  2-5pctCapShortage.csv                          432   
2       5pctCapShort.csv                          721   
3  7-5pctCapShortage.csv                          987   

   Hours with Battery <30%  Hours with Unmet Load  Days with Outages  \
0                      104                     29                  7   
1                      551                    349                 49   
2                      789                    592                 72   
3                     1112                    815                 97   

   Total Outages (Year)  
0                     7  
1                    62  
2                   101  
3                   133  
     month  day  hours_with_unmet_load  #_of_outages                   File
0        5   19                      5             1    0pctCapShortage.csv
1        6   18                      6             1    0pctCapShortage

C:\Users\Nolan Welsh\AppData\Local\Temp\ipykernel_28748\73260671.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  outages_day = unmet_load_df.groupby(['month', 'day']).apply(count_consecutive_groups).reset_index()


In [5]:
results_df


,File,Hours with Capacity Shortage,Hours with Battery <30%,Hours with Unmet Load,Days with Outages,Total Outages (Year)
0,0pctCapShortage.csv,NA,104,29,7,7
1,2-5pctCapShortage.csv,432,551,349,49,62
2,5pctCapShort.csv,721,789,592,72,101
3,7-5pctCapShortage.csv,987,1112,815,97,133


In [6]:
outages_summary_df

,month,day,hours_with_unmet_load,#_of_outages,File
0,5,19,5,1,0pctCapShortage.csv
1,6,18,6,1,0pctCapShortage.csv
2,6,19,4,1,0pctCapShortage.csv
3,8,4,4,1,0pctCapShortage.csv
4,11,9,3,1,0pctCapShortage.csv
...,...,...,...,...,...
220,12,22,13,1,7-5pctCapShortage.csv
221,12,23,6,1,7-5pctCapShortage.csv
222,12,24,12,3,7-5pctCapShortage.csv
223,12,25,7,1,7-5pctCapShortage.csv


Save Summary  results 

In [7]:

summary_path = os.path.join(dir_path, 'outages_summary_results.csv')
results_df.to_csv(summary_path, index=False)

outages_daily_detail_path = os.path.join(dir_path, 'outages_daily_details.csv')
outages_summary_df.to_csv(outages_daily_detail_path, index=False)

